### Decorators

In [4]:
def value_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        res = func(x)
        print('Value', res)
        print("After calling " + func.__name__)
    return function_wrapper

@value_decorator
def succ(n):
    return n + 1

@value_decorator
def random(n):
    return n**2

succ(10)

random(4)

Before calling succ
Value 11
After calling succ
Before calling random
Value 16
After calling random


In [6]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [7]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])
        
waste_some_time(1000)

Finished 'waste_some_time' in 1.9232 secs


In [8]:
import numpy as np

@timer
def waste_some_time_with_numpy(num_times):
    for _ in range(num_times):
        np.power(np.arange(10000), 2).sum()
        
waste_some_time_with_numpy(1000)

Finished 'waste_some_time_with_numpy' in 0.0433 secs


In [9]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

In [10]:
import math

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

approximate_e()

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24
Calling factorial(5)
'factorial' returned 120
Calling factorial(6)
'factorial' returned 720
Calling factorial(7)
'factorial' returned 5040
Calling factorial(8)
'factorial' returned 40320
Calling factorial(9)
'factorial' returned 362880
Calling factorial(10)
'factorial' returned 3628800
Calling factorial(11)
'factorial' returned 39916800
Calling factorial(12)
'factorial' returned 479001600
Calling factorial(13)
'factorial' returned 6227020800
Calling factorial(14)
'factorial' returned 87178291200
Calling factorial(15)
'factorial' returned 1307674368000
Calling factorial(16)
'factorial' returned 20922789888000
Calling factorial(17)
'factorial' returned 355687428096000


2.7182818284590455

In [11]:
@timer
def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

approximate_e()

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24
Calling factorial(5)
'factorial' returned 120
Calling factorial(6)
'factorial' returned 720
Calling factorial(7)
'factorial' returned 5040
Calling factorial(8)
'factorial' returned 40320
Calling factorial(9)
'factorial' returned 362880
Calling factorial(10)
'factorial' returned 3628800
Calling factorial(11)
'factorial' returned 39916800
Calling factorial(12)
'factorial' returned 479001600
Calling factorial(13)
'factorial' returned 6227020800
Calling factorial(14)
'factorial' returned 87178291200
Calling factorial(15)
'factorial' returned 1307674368000
Calling factorial(16)
'factorial' returned 20922789888000
Calling factorial(17)
'factorial' returned 355687428096000
Finished 'approximate_e' in 0.0006 secs


2.7182818284590455

In [12]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [16]:
c = Circle(5)
c.radius

5

In [18]:
c.area

78.5398163375

In [19]:
c.radius = 2
c.area

12.566370614

In [20]:
c.area = 100

AttributeError: can't set attribute

In [22]:
c.cylinder_volume(height=4)

50.265482456

In [23]:
c.radius = -1

ValueError: Radius must be positive

In [24]:
c = Circle.unit_circle()
c.radius

1

In [25]:
c.pi()

3.1415926535

In [26]:
Circle.pi()

3.1415926535

In [28]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [29]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")
    
greet('Cat')

Hello Cat
Hello Cat
Hello Cat
Hello Cat


In [30]:
@timer
@repeat(num_times=6)
def greet(name):
    print(f"Hello {name}")
    
greet('Cat')

Hello Cat
Hello Cat
Hello Cat
Hello Cat
Hello Cat
Hello Cat
Finished 'greet' in 0.0003 secs


In [31]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

In [32]:
@count_calls
def say_whee():
    print("Whee!")

say_whee()

Call 1 of 'say_whee'
Whee!


In [33]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [34]:
say_whee()

Call 3 of 'say_whee'
Whee!


In [35]:
say_whee.num_calls

3

In [36]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")

say_whee()

Call 1 of 'say_whee'
Whee!


In [37]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [38]:
say_whee()

Call 3 of 'say_whee'
Whee!


In [39]:
say_whee.num_calls

3

In [40]:
import functools

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

In [41]:
@singleton
class TheOne:
    pass

In [42]:
first_one = TheOne()
another_one = TheOne()

In [43]:
print(id(first_one))
print(id(another_one))

140309429285904
140309429285904


In [44]:
first_one is another_one

True

In [45]:
class TheOne:
    pass

In [46]:
first_one = TheOne()
another_one = TheOne()

In [47]:
print(id(first_one))
print(id(another_one))

140309428913552
140309428913488


In [48]:
first_one is another_one

False

In [49]:
def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit

import math

@set_unit("cm^3")
def volume(radius, height):
    return math.pi * radius**2 * height

In [52]:
print(volume(3, 5), volume.unit)

141.3716694115407 cm^3


In [53]:
@set_unit("cm^2")
def area(height, width):
    return height * width

In [54]:
print(area(3, 4), area.unit)

12 cm^2


In [64]:
print(area(3, 4) + volume(3, 5), area.unit + volume.unit)

153.3716694115407 cm^2cm^3


In [55]:
def add_unit(a, b):
    if a.unit == b.unit:
        return a(2, 3) + b(3, 5)
    else:
        raise ValueError("Wrong units!")

In [56]:
add_unit(area, area)

21

In [57]:
add_unit(area, volume)

ValueError: Wrong units!

In [58]:
def polynomial_creator(a, b, c):
    def polynomial(x):
        return a * x**2 + b * x + c
    return polynomial

In [59]:
p1 = polynomial_creator(2, 3, -1)
p2 = polynomial_creator(-1, 2, 1)

for x in range(-2, 2, 1):
    print(f"Arg: {x}, p1(Arg): {p1(x)}, p2(Arg): {p2(x)}")

Arg: -2, p1(Arg): 1, p2(Arg): -7
Arg: -1, p1(Arg): -2, p2(Arg): -2
Arg: 0, p1(Arg): -1, p2(Arg): 1
Arg: 1, p1(Arg): 4, p2(Arg): 2


In [60]:
print(polynomial_creator)

<function polynomial_creator at 0x7f9c56522a70>


In [61]:
print(p1)

<function polynomial_creator.<locals>.polynomial at 0x7f9c55acd200>


In [75]:
def polynomial_creator(*coefficients):
    """ coefficients are in the form a_n, ... a_1, a_0 
    """
    def polynomial(x):
        res = 0
        for index, coeff in enumerate(coefficients[::-1]):
            res += coeff * x** index
        return res
    return polynomial

In [76]:
p1 = polynomial_creator(4)
p2 = polynomial_creator(2, 4)
p3 = polynomial_creator(1, 8, -1, 3, 2)
p4  = polynomial_creator(-1, 2, 1)

for x in range(-2, 2, 1):
    print(x, p1(x), p2(x), p3(x), p4(x))

-2 4 0 -56 -7
-1 4 2 -9 -2
0 4 4 2 1
1 4 6 13 2
